<h1>Extracting and Visualizing Stock Data</h1>

In [62]:
import yfinance as yf
import pandas as pd
import requests
from bs4 import BeautifulSoup
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [63]:
import plotly.io as pio
# Will render inside an iframe
pio.renderers.default = "iframe"

In [64]:
import warnings
# Ignore all warnings
warnings.filterwarnings("ignore", category=FutureWarning)

Will define the function `make_graph`. Which takes a dataframe with stock data, a dataframe with revenue data (dataframe contains Date and Revenue columns), and the name of the stock.

In [58]:
def make_graph(stock_data, revenue_data, stock):
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, subplot_titles=("Historical Share Price", "Historical Revenue"), vertical_spacing=.3)

    stock_data_specific = stock_data[stock_data.Date <= '2021-06-14']
    revenue_data_specific = revenue_data[revenue_data.Date <= '2021-04-30'].copy()

    fig.add_trace(go.Scatter(
        x=pd.to_datetime(stock_data_specific.Date),
        y=stock_data_specific.Close.astype("float"),
        name="Share Price"
    ), row=1, col=1)

    # Clean revenue before plotting
    revenue_data_specific["Revenue"] = (
        revenue_data_specific["Revenue"]
        .str.replace("$", "", regex=False)
        .str.replace(",", "", regex=False)
    )
    revenue_data_specific["Revenue"] = pd.to_numeric(revenue_data_specific["Revenue"], errors="coerce")

    fig.add_trace(go.Scatter(
        x=pd.to_datetime(revenue_data_specific.Date),
        y=revenue_data_specific.Revenue,
        name="Revenue"
    ), row=2, col=1)

    fig.update_xaxes(title_text="Date", row=1, col=1)
    fig.update_xaxes(title_text="Date", row=2, col=1)
    fig.update_yaxes(title_text="Price ($US)", row=1, col=1)
    fig.update_yaxes(title_text="Revenue ($US Millions)", row=2, col=1)

    fig.update_layout(showlegend=False, height=900, title=stock, xaxis_rangeslider_visible=True)
    fig.show()

<h3>Using yfinance to Extract Stock Data</h3>

First using the `Ticker` function and ticker symbol of the stock, we need to extract data on to create a ticker object. The stock is Tesla so, its ticker symbol is `TSLA`.

In [65]:
tesla = yf.Ticker('TSLA')

Then, using the ticker object and the function `history` to extract stock information and save it in a dataframe named `tesla_data`. Will set the `period` parameter to ` "max" ` so we get information for the maximum amount of time.


In [48]:
tesla_data = tesla.history(period='max')

Will **Reset the index** using the `reset_index(inplace=True)` function on the tesla_data DataFrame.


In [49]:
tesla_data.reset_index(inplace=True)
tesla_data.head()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2010-06-29 00:00:00-04:00,1.266667,1.666667,1.169333,1.592667,281494500,0.0,0.0
1,2010-06-30 00:00:00-04:00,1.719333,2.028000,1.553333,1.588667,257806500,0.0,0.0
2,2010-07-01 00:00:00-04:00,1.666667,1.728000,1.351333,1.464000,123282000,0.0,0.0
3,2010-07-02 00:00:00-04:00,1.533333,1.540000,1.247333,1.280000,77097000,0.0,0.0
4,2010-07-06 00:00:00-04:00,1.333333,1.333333,1.055333,1.074000,103003500,0.0,0.0


<h3>Using Webscraping to Extract Tesla Revenue Data</h3>

Useing the `requests` library to download the webpage:

https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0220EN-SkillsNetwork/labs/project/revenue.htm

In [50]:
url = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0220EN-SkillsNetwork/labs/project/revenue.htm'
html_data = requests.get(url).text

Parsing the html data using `beautiful_soup`, using parser (`html.parser`).

In [51]:
import lxml

soup = BeautifulSoup(html_data, 'html.parser')

Using `BeautifulSoup`function to extract the table with `Tesla Quarterly Revenue` and store it into a dataframe named `tesla_revenue`.

In [52]:
tesla_revenue = pd.DataFrame(columns=['Date', 'Revenue'])

for table in soup.find_all('table'):
    if "Tesla Quarterly Revenue" in table.find('th').text:
        for row in table.find("tbody").find_all("tr"):
            col = row.find_all("td")
            date = col[0].text
            revenue = col[1].text
            tesla_revenue = pd.concat([tesla_revenue, pd.DataFrame({"Date":[date], "Revenue":[revenue]})], ignore_index=True)

In [53]:
#to remove comma and dollar sign in the Revenue column
tesla_revenue["Revenue"] = tesla_revenue['Revenue'].str.replace(',$',"",regex=True)

#to remove an null or empty strings in the Revenue column
tesla_revenue.dropna(inplace=True)

tesla_revenue = tesla_revenue[tesla_revenue['Revenue'] != ""]

<h3>Plotting the Tesla Stock Graph using make_graph function</h3>


In [55]:
tesla_revenue.tail()

,Date,Revenue
48,2010-09-30,$31
49,2010-06-30,$28
50,2010-03-31,$21
52,2009-09-30,$46
53,2009-06-30,$27


In [59]:
make_graph(tesla_data, tesla_revenue, 'Tesla')

<h3>Final Thoughts</h3>

This project helped me practice end-to-end data analysis using Python. 
From cleaning the dataset to visualizing insights, each step clarified how data transforms into something meaningful.

**Tools Used:** Python, Pandas, NumPy, Matplotlib  
**Skills Practiced:** Data cleaning, EDA, grouping, filtering, visualization, exporting results.

I'll keep refining these skills as I move closer to a data analyst role.